In [ ]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
import xgboost as xgb
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.graphics.api as smg
from warnings import filterwarnings
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from graphs.utils import plot_morris_method_graph
from analysis.utils import get_sa_problem
from SALib.sample.morris import sample
from SALib.analyze.morris import analyze
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

filterwarnings('ignore')

In [ ]:
CV = ShuffleSplit(100)
SCORING = 'neg_mean_squared_error'

# Data Initialization

In [ ]:
DATASET_PROPERTIES_PATH = r'..\..\results\dataset_properties.csv'
IMPURITY_DIFF_PATH = r'..\..\results\impurity_differences.csv'

In [ ]:
dataset_properties_df = pd.read_csv(DATASET_PROPERTIES_PATH)
dataset_properties_df

In [ ]:
impurity_differences_df = pd.read_csv(IMPURITY_DIFF_PATH)
impurity_differences_df.head()

In [ ]:
data_df = impurity_differences_df.merge(dataset_properties_df, on='name')
data_df = data_df.sample(frac=1)

data_df['impurity_mean_diff'] = data_df['avg_lab_impurity'] - data_df['avg_unlab_impurity']
data_df['impurity_var_sum'] = data_df['var_lab_impurity'] + data_df['var_unlab_impurity']

X_df = data_df.drop(['name', 'impurity_mean_diff', 'impurity_var_sum', 
                     'avg_lab_impurity', 'avg_unlab_impurity', 
                     'var_lab_impurity', 'var_unlab_impurity'], axis=1)

y_df = data_df[['impurity_mean_diff']]

# Data Plots

In [ ]:
corr_scores_df = dataset_properties_df[['silhouette_score', 'calinski_harabasz_score', 'davies_bouldin_score']]
corr_scores_df['custom_score'] = (dataset_properties_df['inter_cluster_spread'] 
                                  / dataset_properties_df['intra_cluster_spread']).reset_index(drop=True)
corr_scores_df = corr_scores_df[corr_scores_df['custom_score'] != np.inf]

corr_scores_matrix = np.corrcoef(corr_scores_df.T)
smg.plot_corr(corr_scores_matrix, corr_scores_df.columns)
plt.show()

In [ ]:
corr_df = pd.concat([X_df, y_df], axis=1).reset_index(drop=True)
corr_matrix_base = np.corrcoef(corr_df.T)
smg.plot_corr(corr_matrix_base, corr_df.columns)
plt.show()

# Sensitivity Analysis

In [ ]:
param_grid = {
    'n_estimators': range(1, 20),
    'max_features': range(1, X_df.shape[1] + 1),
}
random_forest_gs = GridSearchCV(RandomForestRegressor(), 
                                param_grid=param_grid, scoring=SCORING,
                                cv=CV, verbose=True, n_jobs=-1)

random_forest_gs.fit(X_df, y_df);

In [ ]:
param_grid = {
    'max_depth': range(1, 20),
    'num_round': range(5, 100, 5),
}

boosted_trees_gs = GridSearchCV(xgb.XGBRegressor(), 
                      param_grid=param_grid, scoring=SCORING,
                      cv=CV, verbose=True, n_jobs=-1)

boosted_trees_gs.fit(X_df, y_df);

In [ ]:
problem = get_sa_problem(X_df)
inputs = sample(problem, 1000, num_levels=4)

X_morris_df = pd.DataFrame(inputs, columns=X_df.columns)
results = boosted_trees_gs.predict(X_morris_df)

sensitivity_indices = \
    analyze(problem,
            inputs,
            results,
            conf_level=0.95,
            num_levels=5)

plot_morris_method_graph(sensitivity_indices, 'Impurity Differences')
plt.show()

# Lasso

In [ ]:
pipe = make_pipeline(StandardScaler(), Lasso())

param_grid = {'lasso__alpha': np.logspace(-4, 2, 100)}
lasso_gs = GridSearchCV(pipe, 
                        param_grid=param_grid, scoring=SCORING,
                        cv=CV, verbose=True, n_jobs=-1)

lasso_gs.fit(X_df, y_df)
lasso_gs.score(X_df, y_df)

In [ ]:
data = {
    'coef_names': X_df.columns,
    'coef_values': lasso_gs.best_estimator_['lasso'].coef_,
}

coef_df = pd.DataFrame(data)
coef_df

# OLS

In [ ]:
X_linear_df = X_df[['provided_labels_count', 'token_count_avg_entropy_a1']].assign(intercept=1)
X_linear_df['token_count_avg_entropy_a1^2'] = X_df['token_count_avg_entropy_a1'] ** 2
X_linear_df['provided_labels_count*token_count_avg_entropy_a1'] = \
    X_df['provided_labels_count'] * X_df['token_count_avg_entropy_a1']

In [ ]:
ols = sm.OLS(y_df, X_linear_df)
results = ols.fit()
results.summary()

# Model Comparisons

In [ ]:

boosted_trees_gs_scores = cross_val_score(boosted_trees_gs.best_estimator_, 
                                          X_df, y_df, cv=CV, 
                                          scoring=SCORING, n_jobs=-1)
random_forest_gs_scores = cross_val_score(random_forest_gs.best_estimator_, 
                                          X_df, y_df, cv=CV, 
                                          scoring=SCORING, n_jobs=-1)
lin_reg_scores = cross_val_score(LinearRegression(), 
                                 X_linear_df, y_df, cv=CV, 
                                 scoring=SCORING, n_jobs=-1)

In [ ]:
print('Boosted Trees CV NMSE: {:16.6f} (±{:.6f})'.format(boosted_trees_gs_scores.mean(), boosted_trees_gs_scores.std() * 2))
print('Random Forest CV NMSE: {:16.6f} (±{:.6f})'.format(random_forest_gs_scores.mean(), random_forest_gs_scores.std() * 2))
print('Linear Regression CV NMSE: {:12.6f} (±{:.6f})'.format(lin_reg_scores.mean(), lin_reg_scores.std() * 2))

In [ ]:
boosted_trees_gs_total_r2 = boosted_trees_gs.best_estimator_.fit(X_df, y_df).score(X_df, y_df)
random_forest_gs_total_r2 = random_forest_gs.best_estimator_.fit(X_df, y_df).score(X_df, y_df)
lin_reg_total_r2 = LinearRegression().fit(X_linear_df, y_df).score(X_linear_df, y_df)

In [ ]:
print('Boosted Trees Total R^2: {:13.6f}'.format(boosted_trees_gs_total_r2))
print('Random Forest Total R^2: {:13.6f}'.format(random_forest_gs_total_r2))
print('Linear Regression Total R^2: {:9.6f}'.format(lin_reg_total_r2))

# Validity Checks

In [ ]:
X_linear_vif_df = X_linear_df[['provided_labels_count', 'token_count_avg_entropy_a1']]
p = X_linear_vif_df.shape[1]
vif_df = pd.DataFrame()
vif_df['VIF Factor'] = [vif(X_linear_vif_df.values, i) for i in range(p)]
vif_df['features'] = X_linear_vif_df.columns
vif_df

In [ ]:
sm.qqplot(results.resid, line='s')
plt.grid()
plt.plot()
_, p = stats.shapiro(results.resid)
print('Shapiro-Wilk test p-value: {}'.format(p))

In [ ]:
results.resid
plt.scatter(y_df, results.resid)
plt.xlabel('Response')
plt.ylabel('Residual')
plt.grid()
plt.show()